<h2 style="color:#ff6961"> Works in Progress </h2> <ol  style="color:#A7C7E7">
   
  <li>Stratify the data better (by LOB in particular) to better relect realistic scenarios</li>
  <li>Example - for commercial property setting the value of the individual site, use external data</li>

  <li>Not worthwhile spending time here Going between rpy2 (using Python in R, seperate notebooks for now) - Easier in Data bricks %R, %sql, %Python magic commands</li>
  <li>Include some outliers for testing (so can create a control table to flag or drop these fields)</li>


</ol> 

<h2 style="color:#D198B7">Read in Engine to support custom made packages </h2>

In [1]:
source("C:/Users/alexz/OneDrive/Documents/Git/Engine/Engine.R")

<h3 style="color:#98D1B2 ">Load in required libraries</h3>

In [2]:
# List of packages required for the project
packs <- c('dplyr', 'tidyr', 'data.table', 'lubridate', 'ids', 'randgeo', 'tidygeocoder', 'reticulate')

In [3]:
install_load_packages(packs)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'data.table'


The following objects are masked from 'package:dplyr':

    between, first, last



Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




$dplyr
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

$tidyr
[1] "tidyr"     "dplyr"     "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     

$data.table
 [1] "data.table" "tidyr"      "dplyr"      "stats"      "graphics"  
 [6] "grDevices"  "utils"      "datasets"   "methods"    "base"      

$lubridate
 [1] "lubridate"  "data.table" "tidyr"      "dplyr"      "stats"     
 [6] "graphics"   "grDevices"  "utils"      "datasets"   "methods"   
[11] "base"      

$ids
 [1] "ids"        "lubridate"  "data.table" "tidyr"      "dplyr"     
 [6] "stats"      "graphics"   "grDevices"  "utils"      "datasets"  
[11] "methods"    "base"      

$randgeo
 [1] "randgeo"    "ids"        "lubridate"  "data.table" "tidyr"     
 [6] "dplyr"      "stats"      "graphics"   "grDevices"  "utils"     
[11] "datasets"   "methods"    "base"      

$tidygeocoder
 [1] "tidygeocoder" "randgeo"      "ids"          "lubridate"    "data.table"  
 [6] "tidyr"        "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"        

$reticulate
 [1] "reticulate"   "tidygeocoder" "randgeo"      "ids"          "lubridate"   
 [6] "data.table"   "tidyr"        "dplyr"        "stats"        "graphics"    
[11] "grDevices"    "utils"        "datasets"     "methods"      "base"

In [8]:
# Set Sample Size
# Set low since reverse geocoding takes a LONG time without for example a google API
sample_size = 1000

<h3 style="color:#98D1B2 ">Create Dummy Data - DataFrame</h3>

In [9]:
# Create Vector of factors with colNames
colNames = c("Customer_ID", "Purchase_Date", "Cover_Start_Date", "LOB", "Sale_Flag", "Purchase_Price",
             "Claims_Count", "Period_of_Cover", "Premium", "Age", "Broker")

# Create a Empty DataFrame with 0 rows and n columns (set in sample_size)
df = data.frame(matrix(nrow = sample_size, ncol = length(colNames))) 

In [10]:
# Assign column names
colnames(df) = colNames

<h3 style="color:#98D1B2 ">Create Dummy Data - populate DataFrame</h3>

In [11]:
# Generate Random Customer IDs
df$Customer_ID <- random_id(sample_size)

In [12]:
# SET date components:
s_year <- 2022
s_month <- 01 
s_day <- 01

dt_start <- paste(s_year, s_month, s_day, sep = '-')
dt_start <- as.Date(ymd(dt_start))

current_dt <- as.Date(today())

td <- current_dt - dt_start #time difference

In [13]:
date_range <- as.Date('2022-01-01') - as.Date('2023-01-01') # date Range
date_vec <- as.Date('2023-01-01') + sample(0:td, sample_size, replace = TRUE) # vector of dates

df$Purchase_Date <- date_vec
df$Cover_Start_Date = date_vec + sample(1:365, replace = TRUE) # start dates up to a year of purchase date - random

Warning message in unclass(e1) + unclass(e2):
"longer object length is not a multiple of shorter object length"


In [14]:
# Lines of Business
lob_list <- c('Commerical Property', 'Motor', 'Home', 'Gadget', 'Pet', 'Travel')
df$LOB <- sample(lob_list, sample_size, replace=TRUE)

In [15]:
# Sale Flag
df$Sale_Flag <- sample(0:1, sample_size, replace=TRUE)

In [16]:
# Purchase Price
df$Purchase_Price <- sample(10:100000, sample_size, replace=TRUE)

In [17]:
# Premium
df$Premium <- sample(10:1000, sample_size, replace=TRUE)

In [18]:
# Claims Count
# If there is a sale - there is a possibility of a claim - otherwise it must default to zero
df$Claims_Count <- ifelse(df$Sale_Flag == 1, 
                      sample(0:1, sample_size, replace = T),
                      0)

In [19]:
# Period of cover by months
sample_input <- c(12, 24, 36, 48)
df$Period_of_Cover <- sample(sample_input, sample_size, TRUE)

In [20]:
# Lats & Longs - can then Reverse Geocode to get Postcode
lats_longs <- rg_position(sample_size)
lats_longs <- as.data.frame(t(data.frame(lats_longs)))

names(lats_longs) <- c('Latitude', 'Longitude')

df$Latitude <- lats_longs$Latitude
df$Longitude <- lats_longs$Longitude

In [23]:
# Age
df$Age <- sample(18:80, sample_size, replace=TRUE)

In [24]:
# Lines of Business
broker_list <- c('london_ins', 'some_syndicate', 'some_mga')
df$Broker <- sample(broker_list, sample_size, replace=TRUE)

In [21]:
View(head(df))

Customer_ID                      Purchase_Date Cover_Start_Date LOB  
1 d2d2062d6d73bfe37440b5bc9aebc518 2023-08-04    2024-01-15       Motor
2 ef5865f3cbf1d48b814e9716f235cfbf 2023-03-30    2023-12-02       Motor
3 5a30397fc844734533879fef33313971 2023-11-28    2024-07-11       Motor
4 61474708950929d053ef8a5cb3ca5c8e 2023-10-02    2024-05-08       Motor
5 afc58a586eeae1560526c07651f6bd80 2023-04-30    2024-04-12       Motor
6 7fdcdd9fd1a203c40e5743b88132ff99 2023-04-18    2023-04-28       Motor
  Sale_Flag Purchase_Price Claims_Count Period_of_Cover Premium Age Broker
1 0         16898          0            48              802     NA  NA    
2 0         21716          0            12              155     NA  NA    
3 1         52179          1            12              503     NA  NA    
4 1         61232          1            48              804     NA  NA    
5 0          2711          0            24               94     NA  NA    
6 0         17872          0            24              665     NA  NA    
  Latitude   Longitude   
1 -115.15198 -58.22413537
2 -155.84526 -55.21116345
3 -131.15406  17.26387609
4   27.15229 -20.92771000
5  -10.96663  17.87203882
6   24.09823   0.07360046

In [22]:
# get post_codes 
# if not in the arctic or lake Victoria
# very slow - free but time cost
reverse <- lats_longs %>%
  reverse_geocode(lat = Latitude, long = Longitude, method = 'osm',
                  address = address_found, full_results = TRUE)

Passing 519 coordinates to the Nominatim single coordinate geocoder

Query completed in: 528.6 seconds



In [25]:
View(head(df))

Customer_ID                      Purchase_Date Cover_Start_Date LOB  
1 d2d2062d6d73bfe37440b5bc9aebc518 2023-08-04    2024-01-15       Motor
2 ef5865f3cbf1d48b814e9716f235cfbf 2023-03-30    2023-12-02       Motor
3 5a30397fc844734533879fef33313971 2023-11-28    2024-07-11       Motor
4 61474708950929d053ef8a5cb3ca5c8e 2023-10-02    2024-05-08       Motor
5 afc58a586eeae1560526c07651f6bd80 2023-04-30    2024-04-12       Motor
6 7fdcdd9fd1a203c40e5743b88132ff99 2023-04-18    2023-04-28       Motor
  Sale_Flag Purchase_Price Claims_Count Period_of_Cover Premium Age
1 0         16898          0            48              802     27 
2 0         21716          0            12              155     23 
3 1         52179          1            12              503     64 
4 1         61232          1            48              804     28 
5 0          2711          0            24               94     36 
6 0         17872          0            24              665     47 
  Broker         Latitude   Longitude   
1 london_ins     -115.15198 -58.22413537
2 some_syndicate -155.84526 -55.21116345
3 london_ins     -131.15406  17.26387609
4 some_mga         27.15229 -20.92771000
5 london_ins      -10.96663  17.87203882
6 some_syndicate   24.09823   0.07360046

<h2 style="color:#ff6961"> Works in Progress For Random Address Generation (US ONLY) </h2> <ol  style="color:#A7C7E7">
   
  <li>Option 1 use the package reticulate (and for the moment manually input the sample_size) </li>
  <li>Option 2 update the python code so it can be called by a system call to the cmd</li>
  <li>At the moment both options mean writing the data from python to the data folder & recalling it back in via R (read.csv)</li>

</ol> 

In [26]:
library(reticulate)

In [27]:
# Sample Size
sample_size

[1] 1000

In [ ]:
#source_python('random_address_py.py')

In [28]:
py_run_string("
# pip install random-address
# pip install pandas

import os
import pandas as pd
import random_address

from random_address import real_random_address

# Generate random addresses in the USA

def create_random_address_us():
    
    add_dict = real_random_address()
    add_df = pd.json_normalize(add_dict)
    
    return (add_df)

# Create Schema

df= pd.DataFrame({'address1': [],
                 'address2': [],
                 'city': [],
                 'state': [],
                 'postalCode': [],
                 'coordinates.lat': [],
                 'coordinates.lng': []
                 })

# Change Dir to Data Folder
os.chdir('C:/Users/alexz/OneDrive/Documents/AGCS/Data')

# Create csv if not exist
filename = 'random_us_addresses.csv'
if not os.path.isfile(filename):
    df.to_csv(filename, header='column_names', index=False)
else:  # else it exists so append without writing the header
    df.to_csv(filename, mode='a', header=False, index=False)

for i in range(5):
    address_data = create_random_address_us()
    # Write data to csv file
    address_data.to_csv(filename, mode='a', header=False, index=False)
")

In [ ]:
appended_data

In [5]:
library(reticulate)

py_evaluate <- function(code) {
  builtins <- import_builtins(convert = TRUE)
  globals <- py_eval("globals()", convert = FALSE)
  locals <- globals
  parsed <- builtins$compile(code, "<string>", "single")
  builtins$eval(parsed, globals, locals)
}

py_evaluate("2")